# LDA에서 Dynamic Topic Model

<img src="figs/dtm_fig.png">

* LDA를 time slice 단위로 여러개를 붙인 형태
* 토픽 분포에 해당하는 파라미터인 Beta값을 time slice가 넘어갈때 넘겨주는 것으로 시간대별 토픽 분포를 반영하게 함

<img src="figs/dtm_fig2.png">

### 1. 대화내용 불러오기

In [1]:
import pandas as pd
import numpy as np
import pickle
from pprint import pprint
import re

In [2]:
with open("./cleaned_data.pk", "rb") as f:
    data = pickle.load(f)

data.reset_index(drop=True, inplace=True)
print(data.head())
print(data.info())

         Date   Speaker timetype   time contents
0  2021-06-14  김태환 형 17       오전   6:01      고민중
1  2021-06-14    이현직 16       오전   8:00      셤잘쳐
2  2021-06-14  김태환 형 17       오전   8:00      귀여워
3  2021-06-14  김태환 형 17       오전  10:07       시발
4  2021-06-14  김태환 형 17       오전  10:08   담배가 쓰다
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 980 entries, 0 to 979
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Date      980 non-null    object
 1   Speaker   980 non-null    object
 2   timetype  980 non-null    object
 3   time      980 non-null    object
 4   contents  980 non-null    object
dtypes: object(5)
memory usage: 38.4+ KB
None


### 2. 분석시기 설정하기

In [3]:
# 시간정보 열 datetime 정보로 변환
data["Date"] = pd.to_datetime(data["Date"])
print(data["Date"])

# 인덱스 넣기
data = data.set_index("Date")
data.head()

0     2021-06-14
1     2021-06-14
2     2021-06-14
3     2021-06-14
4     2021-06-14
         ...    
975   2021-06-30
976   2021-06-30
977   2021-06-30
978   2021-06-30
979   2021-06-30
Name: Date, Length: 980, dtype: datetime64[ns]


,Speaker,timetype,time,contents
Date,,,,
2021-06-14,김태환 형 17,오전,6:01,고민중
2021-06-14,이현직 16,오전,8:00,셤잘쳐
2021-06-14,김태환 형 17,오전,8:00,귀여워
2021-06-14,김태환 형 17,오전,10:07,시발
2021-06-14,김태환 형 17,오전,10:08,담배가 쓰다


#### 연도별 대화내용 분리 (DTM 분석을 위해)

In [4]:
# 지정된 시기별로 분리
day10 = data["2021-06-10" : "2021-06-19"]
day20 = data["2021-06-20" : "2021-06-29"]
day30 = data["2021-06-30" : "2021-06-30"]

In [5]:
print(day10.head())
print(day20.head())
print(day30.head())

             Speaker timetype   time contents
Date                                         
2021-06-14  김태환 형 17       오전   6:01      고민중
2021-06-14    이현직 16       오전   8:00      셤잘쳐
2021-06-14  김태환 형 17       오전   8:00      귀여워
2021-06-14  김태환 형 17       오전  10:07       시발
2021-06-14  김태환 형 17       오전  10:08   담배가 쓰다
             Speaker timetype  time  \
Date                                  
2021-06-21  김태환 형 17       오후  2:53   
2021-06-21  김태환 형 17       오후  2:53   
2021-06-21  김표선 형 16       오후  3:06   
2021-06-21  김표선 형 16       오후  3:06   
2021-06-21  김태환 형 17       오후  3:12   

                                              contents  
Date                                                    
2021-06-21  위상 진짜 takehome 시험인데 30점만점에 평균이 어떻게 17점이나오지  
2021-06-21                     구글링도 허용인데 이걸 던지는 놈들이 있네  
2021-06-21                                       17점이면  
2021-06-21                                     잘나온거아니냐  
2021-06-21                     구글이랑 강의노트 보면 반 이상은 나오는데  
     

In [6]:
print(day10.info())
print(day20.info())
print(day30.info())

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 723 entries, 2021-06-14 to 2021-06-18
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Speaker   723 non-null    object
 1   timetype  723 non-null    object
 2   time      723 non-null    object
 3   contents  723 non-null    object
dtypes: object(4)
memory usage: 28.2+ KB
None
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 206 entries, 2021-06-21 to 2021-06-29
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Speaker   206 non-null    object
 1   timetype  206 non-null    object
 2   time      206 non-null    object
 3   contents  206 non-null    object
dtypes: object(4)
memory usage: 8.0+ KB
None
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 51 entries, 2021-06-30 to 2021-06-30
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Spea

> 데이터를 대충 균등하게 수를 맞춰주는게 좋음.

##### 시기별로 나눠서 데이터 저장하기

In [7]:
# time_slice별로 데이터 갯수를 저장할 리스트 생성
time_slice = []

# 시기별 데이터 가져오기
slice10 = list(day10["contents"])
slice20 = list(day20["contents"])
slice30 = list(day30["contents"])

# LDA 분석할 때와 비슷하게 모든 분석 대상 텍스트를 담는 리스트 생성
tokenized_data = [msg.split() for msg in (slice10 + slice20 + slice30)]

# 각 slice에 들어 있는 갯수를 원소로 갖는 리스트 생성
time_slice.append(len(slice10))
time_slice.append(len(slice20))
time_slice.append(len(slice30))

In [8]:
print(len(tokenized_data))
print(time_slice)
print(slice10[:5])
print(slice20[:5])
print(slice30[:5])
pprint(tokenized_data[:5])

980
[723, 206, 51]
['고민중', '셤잘쳐', '귀여워', '시발', '담배가 쓰다']
['위상 진짜 takehome 시험인데 30점만점에 평균이 어떻게 17점이나오지', '구글링도 허용인데 이걸 던지는 놈들이 있네', '17점이면', '잘나온거아니냐', '구글이랑 강의노트 보면 반 이상은 나오는데']
['고토 비쁠', '만족한다 ', ' 고토 비쁠은 잘한거지', '아  락 내가 어이스크림 함 살게', '동현이 빨리 까라']
[['고민중'], ['셤잘쳐'], ['귀여워'], ['시발'], ['담배가', '쓰다']]


### 3. Dynamic Topic Model 돌리기

In [9]:
from gensim.models import ldaseqmodel
#from gensim.models import CoherenceModel
from gensim.corpora import Dictionary, bleicorpus
#from gensim.matutils import hellinger
from gensim import corpora
#from tqdm import tqdm_notebook
#from time import time

import os

C:\Users\colin\anaconda3\envs\DH\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


#### dictionary와 corpus 만들기

In [10]:
# DTM dictionary 저장.
if not os.path.exists("kakao(DTM)_dict"):
    dictionary = corpora.Dictionary(tokenized_data)
    dictionary.save("kakao(DTM)_dict")
else:
    dictionary = Dictionary.load("kakao(DTM)_dict")


# DTM Corpus 저장.
if not os.path.exists('kakao(DTM)_corpus'):
    corpus = [dictionary.doc2bow(doc) for doc in tokenized_data]
    corpora.BleiCorpus.serialize('kakao(DTM)_corpus', corpus)
else:
    corpus = bleicorpus.BleiCorpus('kakao(DTM)_corpus')

### Run DTM model 

In [11]:
# 주의! 매우매우매우 오래걸릴지도 모릅니다... 시간과 여유가 있을때 해보세요!
NUM_TOPICS = 4

if not os.path.exists("kakao(DTM)_model"):
    dtm_model = ldaseqmodel.LdaSeqModel(corpus=corpus, id2word=dictionary, time_slice=time_slice, num_topics=NUM_TOPICS, passes=10)
    dtm_model.save("kakao(DTM)_model")
else:
    dtm_model = ldaseqmodel.LdaSeqModel.load("kakao(DTM)_model")

C:\Users\colin\anaconda3\envs\DH\lib\site-packages\gensim\models\ldaseqmodel.py:1478: RuntimeWarning: invalid value encountered in double_scalars
  converged = np.fabs((lhood_old - lhood) / (lhood_old * total))
C:\Users\colin\anaconda3\envs\DH\lib\site-packages\gensim\models\ldaseqmodel.py:297: RuntimeWarning: divide by zero encountered in double_scalars
  convergence = np.fabs((bound - old_bound) / old_bound)
C:\Users\colin\anaconda3\envs\DH\lib\site-packages\gensim\models\ldaseqmodel.py:1478: RuntimeWarning: invalid value encountered in double_scalars
  converged = np.fabs((lhood_old - lhood) / (lhood_old * total))
C:\Users\colin\anaconda3\envs\DH\lib\site-packages\gensim\models\ldaseqmodel.py:1478: RuntimeWarning: invalid value encountered in double_scalars
  converged = np.fabs((lhood_old - lhood) / (lhood_old * total))
C:\Users\colin\anaconda3\envs\DH\lib\site-packages\gensim\models\ldaseqmodel.py:1478: RuntimeWarning: invalid value encountered in double_scalars
  converged = np.f

##### DTM 결과 보기

In [12]:
# 고정된 시간 내에서 전체 토픽보기
pprint(dtm_model.print_topics(time=0, top_terms=10))

[[('난', 0.011823712585881219),
  ('1', 0.010545052071412401),
  ('아', 0.010523512021600559),
  ('또', 0.010515348964588897),
  ('오늘', 0.01050408021089675),
  ('락원이', 0.009220096564257612),
  ('다', 0.009205337139974337),
  ('락원아', 0.009189636655599),
  ('진짜', 0.009189015661037357),
  ('동현이', 0.007929557569297887)],
 [('락원이', 0.028078132861481944),
  ('태환', 0.014812078984784455),
  ('왜', 0.0131236185239444),
  ('지금', 0.01135175279676454),
  ('존나', 0.009688260078705269),
  ('현직이', 0.008240646227727513),
  ('내일', 0.008232028986059584),
  ('너무', 0.008205419107188543),
  ('이거', 0.006608710212380627),
  ('내', 0.00660871021238061)],
 [('나', 0.02840537133388891),
  ('아', 0.020096978556572113),
  ('난', 0.016984884283323138),
  ('진짜', 0.013934354872377533),
  ('그럼', 0.01083979087921919),
  ('락원', 0.01082762315335926),
  ('아니', 0.010819430730223048),
  ('그냥', 0.010494030807506964),
  ('좀', 0.009559786842434099),
  ('이렇게', 0.009283248599728393)],
 [('니', 0.013866392389156752),
  ('오늘', 0.01382721672

In [13]:
pprint(dtm_model.print_topics(time=1, top_terms=10))

[[('난', 0.011883406853342441),
  ('또', 0.010578819129238432),
  ('오늘', 0.010574312597671168),
  ('아', 0.010542461950995488),
  ('1', 0.01051259047929246),
  ('락원아', 0.009304285572794852),
  ('진짜', 0.009261376778611613),
  ('다', 0.009237878460850281),
  ('락원이', 0.009207075729403017),
  ('동현이', 0.007881525283134563)],
 [('락원이', 0.028222770749637156),
  ('태환', 0.014719087195360607),
  ('왜', 0.013191745121053988),
  ('지금', 0.011317920625142766),
  ('존나', 0.009714836418105782),
  ('너무', 0.008313342417078061),
  ('현직이', 0.00822158265869537),
  ('내일', 0.008216034756395022),
  ('때', 0.006658382596493378),
  ('이제', 0.006655460798184122)],
 [('나', 0.029126169885841033),
  ('아', 0.020101218660737753),
  ('난', 0.01700659179867647),
  ('진짜', 0.013873432743754296),
  ('락원', 0.010807063201045767),
  ('아니', 0.010801622658560734),
  ('그럼', 0.01076482646015651),
  ('그냥', 0.010583894906651344),
  ('좀', 0.00960658207679994),
  ('이렇게', 0.009277256811591042)],
 [('오늘', 0.01386520174354237),
  ('니', 0.013828